# Setting up Docker container with test data

This Notebook will describe how you can set up the dataset used to test the two postgresql configurations

#### Spin up postgres Docker container from `great-expectations` repo

* `great_expectations/Assets/docker/postgresql` will have a `docker-composer.yml` that can be used to spin up a test postgres container

Use `docker-composer up` in this directory to spin up the Docker container. 


In [12]:
# This utility is not for general use. It is only to support testing
def load_data_into_database(
    table_name: str, csv_path: str, connection_string: str
) -> None:
    import pandas as pd
    import sqlalchemy as sa

    engine = sa.create_engine(connection_string)
    try:
        connection = engine.connect()
        print(f"Dropping table {table_name}")
        connection.execute(f"DROP TABLE IF EXISTS {table_name}")
        df = pd.read_csv(csv_path)
        # Improving test performance by only loading the first 10 rows of our test data into the db
        df = df.head(10)
        print(f"Creating table {table_name} from {csv_path}")
        df.to_sql(name=table_name, con=engine, index=False)
    except SQLAlchemyError as e:
        logger.error(
            f"""Docs integration tests encountered an error while loading test-data into test-database."""
        )
        raise
    finally:
        connection.close()
        engine.dispose()


In [13]:
CONNECTION_STRING = "postgresql+psycopg2://postgres:@localhost/test_ci"

In [14]:
load_data_into_database(
    table_name="titanic",
    csv_path="Titanic.csv",
    connection_string=CONNECTION_STRING,
)

Dropping table titanic
Creating table titanic from Titanic.csv
